<a href="https://colab.research.google.com/github/ioanabuze/Data-Engineering_W4/blob/master/Amazon_Review_Weak_Labelling_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Code here to push the Notebook to Git

In [0]:
# Import Libraries, OS and Setup 

%matplotlib inline

from sklearn import datasets
import numpy as np
import pandas as pd

!pip install snorkel

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis


from wordcloud import WordCloud
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np

In [0]:
# pyspark setup

!pip install pyspark
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd "/content/drive/My Drive/Colab Notebooks/Data Engineering Group Project"

[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/Data Engineering Group Project'
/content


In [0]:
import os 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

COLAB_NOTEBOOKS_DATA_PATH="/content/drive/My Drive/Colab Notebooks/Data Engineering Group Project"
SPARK_ARCHIVE='spark-2.4.5-bin-hadoop2.7.tgz'
if SPARK_ARCHIVE not in os.listdir(COLAB_NOTEBOOKS_DATA_PATH):
  !wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

SPARK_ARTIFACTS='spark-2.4.5-bin-hadoop2.7'
if SPARK_ARTIFACTS not in os.listdir(COLAB_NOTEBOOKS_DATA_PATH):
  !tar xf spark-2.4.5-bin-hadoop2.7.tgz

!pip install -q findspark

FileNotFoundError: ignored

In [0]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/My Drive/Colab Notebooks/Data Engineering Group Project/spark-2.4.5-bin-hadoop2.7"

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import findspark
findspark.init()

import pyspark
from pyspark.sql import SQLContext

In [0]:
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [0]:
# Import the Data

amazon_reviews_dataset=sqlContext.read.csv("/content/drive/My Drive/Colab Notebooks/Data Engineering Group Project/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv",header=True)

In [0]:
# Train/Test Split (25%?)
df_train, df_test = amazon_reviews_dataset()

# We pull out the label vectors for ease of use later
Y_test = df_test.label.values

In [0]:
amazon_reviews_dataset.head(5)

[Row(id='AVqVGZNvQMlgsOJE6eUY', dateAdded='2017-03-03T16:56:05Z', dateUpdated='2018-10-25T16:36:31Z', name='"Amazon Kindle E-Reader 6"" Wifi (8th Generation', asins=' 2016)"', brand='B00ZV9PXP2', categories='Amazon', primaryCategories='Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessories,Used:Tablets,E-Readers,E-Readers & Accessories,Computers/Tablets & Networking,Used:Computers Accessories,iPads Tablets,All Tablets,Tablets & E-readers,Computers & Tablets,Amazon,Tablets & eBook Readers', imageURLs='Electronics', keys='https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.neweggimages.com/NeweggImage/ProductImage/A3FA_1_201801081360871160.jpg,https://i.ebayimg.com/thumbs/images/g/N4IAAOSwoA9Zgkso/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/dpkAAOSwfpVZFKHy/s-l200.jpg,http://i.ebayimg.com/thumbs/images/g/PJgAAOSwiDFYPE8h/s-l200.jpg,http://i.ebayimg.com/thumbs/images/g/m38AAOSwblZZEhFQ/s-l200.jpg,htt

In [0]:
# Visualise the 5 different reviews WordCloud and figure other ways to quantify frequency of words
# From - https://towardsdatascience.com/spam-classifier-in-python-from-scratch-27a98ddd8e73

# WorldCloud - Review == 1 
reviewone = ' '.join(list([mails['label']==1]['message']))
reviewone_wc = WordCloud(width=512,height=512).generate(reviewone)

plt.figure(figsize = (10,8), facecolor = 'k')
plt.imshow(reviewone_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

NameError: ignored

In [0]:
# Set voting values.
ABSTAIN = -1
SPAM = 1 
HAM = 0

In [0]:
# Labelling Funtions for the 5 different review tiers

@labeling_function()
def check(x):
    return SPAM if "check" in x.text.lower() else ABSTAIN

    @labeling_function()
def check(x):
    return SPAM if "check" in x.text.lower() else ABSTAIN

    @labeling_function()
def check(x):
    return SPAM if "check" in x.text.lower() else ABSTAIN
    
    @labeling_function()
def check(x):
    return SPAM if "check" in x.text.lower() else ABSTAIN

In [0]:
# Code to apply all our fucntions
lfs = [#all our functions]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

In [0]:
# Performance Evaluation

In [0]:
coverage_check_out, coverage_check = (L_train != ABSTAIN).mean(axis=0)
print(f"check_out coverage: {coverage_check_out * 100:.1f}%")
print(f"check coverage: {coverage_check * 100:.1f}%")